In [1]:
from transformers import BertForSequenceClassification
from transformers import RobertaForSequenceClassification
from transformers import ElectraForSequenceClassification
from transformers import AutoTokenizer, ElectraTokenizer

from torch.nn.functional import sigmoid
from sklearn.metrics import classification_report

import torch
import pandas as pd

#pd.set_option('display.max_colwidth', None)  # Show full column content
pd.set_option('display.max_columns', None)

In [2]:
Bert_model = BertForSequenceClassification.from_pretrained("../../Models/Second_BERT/")
Bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

RoBERTa_model = RobertaForSequenceClassification.from_pretrained("../../Models/RoBERTa/")
RoBERTa_tokenizer = AutoTokenizer.from_pretrained("roberta-base")

ELECTRA_model = ElectraForSequenceClassification.from_pretrained("../../Models/ELECTRA/")
ELECTRA_tokenizer = ElectraTokenizer.from_pretrained("google/electra-base-discriminator")

In [3]:
df_test = pd.read_csv("test_jigsaw_data.csv")
df_test

,id,comment_text,gender_bias,sexual_bias,religion_bias,race_bias,disability_bias
0,660215,We have no obligation to foist our idea of wha...,0,1,0,0,0
1,696994,Putting aside the exaggerated and unrealistic ...,0,0,0,1,0
2,5963512,So when the NEXT plan was released and the pub...,1,0,0,0,0
3,6333555,Which you should normally get from your mother...,0,0,0,0,0
4,5635048,"Sigh. You're such an absolutist, Prog. Unable ...",0,0,1,1,0
...,...,...,...,...,...,...,...
1901,5430472,If this guy is willing to choke and bodyslam a...,0,0,0,0,0
1902,5760984,Now I suppose we will hear a cadre of explaine...,0,0,0,0,0
1903,5751473,"The media uses the term ""manifesto"" to discred...",1,0,0,0,0
1904,627721,"After serving in WWII, my dad was killed in th...",0,0,0,0,0


In [19]:
texts = df_test["comment_text"].tolist()
labels = df_test[["gender_bias", "sexual_bias", "religion_bias", "race_bias", "disability_bias"]].values

bias_counts = df_test[["gender_bias", "sexual_bias", "religion_bias", "race_bias", "disability_bias"]].sum()

bias_counts


gender_bias        524
sexual_bias        151
religion_bias      464
race_bias          278
disability_bias     42
dtype: int64

In [5]:
def predict_multilabel(model, tokenizer, texts, batch_size=8, device="cpu"):
    model.to(device)
    model.eval()

    all_preds = []

    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            encoding = tokenizer(batch_texts, truncation=True, padding=True, return_tensors="pt").to(device)
            outputs = model(**encoding)
            logits = outputs.logits
            probs = sigmoid(logits)
            preds = (probs > 0.5).int().cpu().numpy()
            all_preds.extend(preds)

    return all_preds


In [7]:
bert_preds = predict_multilabel(Bert_model, Bert_tokenizer, texts)
print("BERT done")
roberta_preds = predict_multilabel(RoBERTa_model, RoBERTa_tokenizer, texts)
print("RoBERTa done")
electra_preds = predict_multilabel(ELECTRA_model, ELECTRA_tokenizer, texts)
print("ELECTRA done")

BERT done
RoBERTa done
ELECTRA done


In [ ]:
target_names = ["gender_bias", "sexual_bias", "religion_bias", "race_bias", "disability_bias"]

bert_report = classification_report(labels, bert_preds, target_names=target_names, output_dict=True, zero_division=0)
roberta_report = classification_report(labels, roberta_preds, target_names=target_names, output_dict=True, zero_division=0)
electra_report = classification_report(labels, electra_preds, target_names=target_names, output_dict=True, zero_division=0)

def extract_f1_scores(report_dict, model_name):
    return pd.DataFrame({
        model_name: [report_dict[label]["f1-score"] for label in target_names]
    }, index=target_names)

bert_f1 = extract_f1_scores(bert_report, "BERT_F1")
roberta_f1 = extract_f1_scores(roberta_report, "RoBERTa_F1")
electra_f1 = extract_f1_scores(electra_report, "ELECTRA_F1")

f1_comparison_df = pd.concat([bert_f1, roberta_f1, electra_f1], axis=1)
f1_comparison_df = f1_comparison_df.round(3)

average_row = f1_comparison_df.mean().to_frame().T
average_row.index = ["Average"]
f1_comparison_df = pd.concat([f1_comparison_df, average_row])

f1_comparison_df = f1_comparison_df.round(3)

f1_comparison_df


,BERT_F1,RoBERTa_F1,ELECTRA_F1
gender_bias,0.856,0.842,0.854
sexual_bias,0.922,0.898,0.890
religion_bias,0.792,0.789,0.775
race_bias,0.850,0.846,0.817
disability_bias,0.680,0.733,0.000
Average,0.820,0.822,0.667


In [ ]:
bert_report = classification_report(labels, bert_preds, target_names=target_names, output_dict=True, zero_division=0)
roberta_report = classification_report(labels, roberta_preds, target_names=target_names, output_dict=True, zero_division=0)
electra_report = classification_report(labels, electra_preds, target_names=target_names, output_dict=True, zero_division=0)

def filter_label_metrics(report_dict, model_name):
    return pd.DataFrame(report_dict).T.loc[target_names][["precision", "recall", "f1-score"]].rename(
        columns=lambda c: f"{model_name}_{c}"
    )

bert_df = filter_label_metrics(bert_report, "BERT")
roberta_df = filter_label_metrics(roberta_report, "RoBERTa")
electra_df = filter_label_metrics(electra_report, "ELECTRA")

comparison_df = pd.concat([bert_df, roberta_df, electra_df], axis=1)

comparison_df = comparison_df.round(3)

comparison_df

,BERT_precision,BERT_recall,BERT_f1-score,RoBERTa_precision,RoBERTa_recall,RoBERTa_f1-score,ELECTRA_precision,ELECTRA_recall,ELECTRA_f1-score
gender_bias,0.821,0.893,0.856,0.779,0.916,0.842,0.819,0.891,0.854
sexual_bias,0.910,0.934,0.922,0.895,0.901,0.898,0.922,0.861,0.890
religion_bias,0.780,0.804,0.792,0.743,0.841,0.789,0.777,0.774,0.775
race_bias,0.824,0.878,0.850,0.816,0.878,0.846,0.822,0.813,0.817
disability_bias,0.600,0.786,0.680,0.627,0.881,0.733,0.000,0.000,0.000
